In [ ]:
from __future__ import annotations

import logging
import pathlib

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)


In [13]:
subjects = {
    "Биология": "biology",
    "Физика": "physics",
    "Математика": "math",
    "Информатика": "informatics",
    "Религиоведение": "religion",
    "Английский": "english",
    "Программирование": "programming",
    "История": "history",
    "Литература": "literature",
    "География": "geography",
    "Введение в геометрию": "intro-geometry",
    "Музыка": "music",
    "Физическая культура": "physical-culture",
    "Русский язык": "russian",
    "Обществознание": "sociology",
    "Алгебра": "algebra",
    "Введение в химию": "intro-chemistry",
    "Вероятность и статистика": "statistics",
    "Геометрия": "geometry",
    "Немецкий язык": "german",
}

trimesters = {
    "III триместр": "3",
    "II триместр": "2",
    "I триместр": "1",
}

In [ ]:
def get_new_name(name: str) -> dict[str, str]:
    """Rename a table name to be more readable.

    Args:
        name (str): The name of the table.

    Returns:
        dict[str, str]: A dictionary containing the new name for each column.s

    """
    _, year, grade, subject, trimester = name.split("_")
    return {
        "year": year,
        "grade": grade,
        "subject": subjects[subject],
        "trimester": trimesters.get(trimester),
    }


# Rename tables
dir_name = pathlib.Path("data")

for file in dir_name.iterdir():
    suffix = file.suffix
    name = file.stem

    new_name_dict = get_new_name(name)
    new_name_dict["suffix"] = suffix

    new_name = "{year}_{grade}_{subject}_{trimester}{suffix}".format(
        **new_name_dict,
    )

    new_file = file.with_name(new_name)
    file.rename(new_file)

    logger.info("Renamed %s to %s", file, new_file)

2022-2023_5-2_biology_1


ValueError: not enough values to unpack (expected 5, got 4)

In [58]:
def process_dataframe(df: pd.DataFrame, table_name: str) -> pd.DataFrame:
    """Process a dataframe and returns a processed dataframe.

    Args:
        df (pd.DataFrame): The dataframe to be processed.
        table_name (str): The name of the table.

    Returns:
        pd.DataFrame: The processed dataframe.

    """
    student_ids: list[str] = []
    marks: list[int] = []

    marks_data: list[str] = []
    marks_weight: list[int] = []

    student_ids_column = df.iloc[3:, 0]

    df.iloc[3:, 0].dropna()

    for col in df.columns:
        mark_data = df[col][1]
        mark_weight = df[col][2]

        for i, val in enumerate(df[col][3:]):
            if type(val) in {int, float} and pd.notna(val):
                student_ids.append(student_ids_column[i + 3])
                marks.append(val)
                marks_data.append(mark_data)
                marks_weight.append(mark_weight)

    year, grade, subject, trimester = table_name.split("_")
    grade = grade.split("-")[0]

    result_df = pd.DataFrame(data={"student_id": student_ids, "mark": marks})
    result_df["grade"] = grade
    result_df["subject"] = subject
    result_df["trimester"] = trimester
    result_df["year"] = year
    result_df["marks_data"] = marks_data
    result_df["marks_weight"] = marks_weight

    return result_df.dropna(subset="student_id").reset_index(drop=True)

In [59]:
main_df = pd.DataFrame(
    columns=[
        "student_id",
        "mark",
        "grade",
        "subject",
        "trimester",
        "year",
        "marks_data",
        "marks_weight",
    ],
)

# Get all files in the directory and process them to a dataframe
for file in dir_name.iterdir():
    suffix = file.suffix
    name = file.stem

    dataframe = pd.read_excel(file)
    main_df = pd.concat(
        [main_df, process_dataframe(dataframe, name)],
        ignore_index=True,
    )

In [60]:
exclude_values = [
    "I", "II", "III", "Годовая", "Внутр. экз.",
]
main_df = main_df[~main_df["marks_data"].isin(exclude_values)]

In [61]:
import re

months = [
    "янв",
    "фев",
    "мар",
    "апр",
    "мая",
    "июн",
    "июл",
    "авг",
    "сен",
    "окт",
    "ноя",
    "дек",
]
months_dict = {month: i + 1 for i, month in enumerate(months)}


def get_date(str_date: str) -> tuple[int, int] | None:
    """Convert string date to tuple (int, int).

    Args:
        str_date (str): string date in format "ddmmm"

    Returns:
        tuple[int, int] | None: tuple (int, int) or None if string is not valid

    """
    re_match_number = re.match(r"\d+", str_date)
    if re_match_number is None:
        return None

    letter_match = re.search(r"[а-яА-Я]+", str_date)
    if letter_match is None:
        return None

    int_date_number = int(re_match_number.group())
    month = months_dict[letter_match.group()]
    return int_date_number, month


get_date("12дек")

(12, 12)

In [62]:
import datetime


def get_datetime_date(datetime_string: int, year: str) -> datetime.date | None:
    """Get the date from a datetime string and year.

    Args:
        datetime_string (int): The datetime string.
        year (str): The year.

    Returns:
        datetime.date | None: The date.

    """
    target_year: int
    begin_year, last_year = year.split("-")
    date_number, month = get_date(datetime_string)

    jan_int, dec_int, sep_int = 1, 12, 9
    if (month >= sep_int and month <= dec_int):
        target_year = int(begin_year)
    elif (month >= jan_int and month < sep_int):
        target_year = int(last_year)
    else:
        return None

    return datetime.date(target_year, month, date_number)


get_datetime_date(datetime_string="13дек", year="2022-2023")

datetime.date(2022, 12, 13)

In [63]:
main_df["mark_date"] = main_df.apply(
    lambda row: get_datetime_date(
        datetime_string=row["marks_data"],
        year=row["year"],
    ),
    axis=1,
)
main_df = main_df.drop(columns=["marks_data", "year"])

In [ ]:
main_df["stuent_id"] = main_df["student_id"].astype(str)
main_df["mark"] = main_df["mark"].astype(int)
main_df["grade"] = main_df["grade"].astype(int)
main_df["subject"] = main_df["subject"].astype(str)
main_df["trimester"] = main_df["trimester"].astype(int)
main_df["marks_weight"] = main_df["marks_weight"].astype(str)
main_df["mark_date"] = pd.to_datetime(main_df["mark_date"])

main_df = main_df.reset_index()
main_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10594 entries, 0 to 10593
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   index         10594 non-null  int64         
 1   student_id    10594 non-null  object        
 2   mark          10594 non-null  int64         
 3   grade         10594 non-null  int64         
 4   subject       10594 non-null  object        
 5   trimester     10594 non-null  int64         
 6   marks_weight  10594 non-null  object        
 7   mark_date     10594 non-null  datetime64[ns]
 8   stuent_id     10594 non-null  object        
dtypes: datetime64[ns](1), int64(4), object(4)
memory usage: 745.0+ KB
